In [1]:
import importlib
import os
import sys
import tomllib
from pathlib import Path
from pprint import pprint

import torch
import wandb
from monai.losses import DiceLoss
from monai.metrics import DiceMetric, HausdorffDistanceMetric
from monai.networks.layers import Norm
from monai.networks.nets import UNet
from monai.utils import set_determinism
from torch.utils.data import DataLoader, random_split

from src.datasets.acdc_dataset import ACDCDataset
from src.train import train
from src.transforms import get_transforms
from src.utils import find_optimal_learning_rate, setup_dirs
from src.visualization import visualize_loss_curves, visualize_predictions

sys.path.insert(0, "..")

In [2]:
root_dir = Path(os.getcwd()).parent
data_dir, log_dir, root_out_dir = setup_dirs(root_dir)
data_dir = data_dir / "ACDC" / "database"

with open(root_dir / "config.toml", "rb") as file:
    config = tomllib.load(file)

pprint(config)
batch_size = config["hyperparameters"].get("batch_size", 4)
epochs = config["hyperparameters"].get("epochs", 100)
learning_rate = config["hyperparameters"].get("learning_rate", 1e-5)
percentage_data = config["hyperparameters"].get("percentage_data", 1.0)
validation_split = config["hyperparameters"].get("validation_split", 0.8)

set_determinism(seed=config["hyperparameters"]["seed"])

{'hyperparameters': {'batch_size': 1,
                     'epochs': 100,
                     'learning_rate': 1e-05,
                     'percentage_data': 1.0,
                     'seed': 42,
                     'validation_split': 0.8}}


In [3]:
importlib.reload(sys.modules["src.transforms"])

train_transforms = get_transforms()

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=4,
    # channels=(26, 52, 104, 208, 416),
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    norm=Norm.BATCH,
    # num_res_units=4,
    # dropout=0.5,
).to(device)

loss_function = DiceLoss(to_onehot_y=True, softmax=True)
# TODO: weight decay check
optimizer = torch.optim.Adam(model.parameters())

metrics = {
    "dice": DiceMetric(include_background=False, reduction="mean"),
    "dice_with_background": DiceMetric(include_background=True, reduction="mean"),
    "hausdorff": HausdorffDistanceMetric(include_background=False, reduction="mean"),
    "dice_per_label": DiceMetric(include_background=False, reduction="mean_batch"),
    "dice_per_label_with_background": DiceMetric(include_background=True, reduction="mean_batch"),
    "hausdorff_per_label": HausdorffDistanceMetric(include_background=False, reduction="mean_batch"),
}

In [5]:
full_dataset_loader = DataLoader(
    ACDCDataset(
        data_dir=data_dir, train=True, transform=train_transforms, percentage_data=1
    ),
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
)

# Find the optimal learning rate using the full dataset
# Use the config learning rate as a midpoint.
optimal_learning_rate = find_optimal_learning_rate(
    model=model,
    optimizer=optimizer,
    criterion=loss_function,
    device=device,
    train_loader=full_dataset_loader,
    start_lr=learning_rate / 100,
    end_lr=learning_rate * 100,
    iterations=100,
)

if optimal_learning_rate is None:
    print("Optimal learning rate not found, using default learning rate.")
    optimal_learning_rate = learning_rate
else:
    print(f"Optimal learning rate found: {optimal_learning_rate}")

for group in optimizer.param_groups:
    group["lr"] = optimal_learning_rate

config["hyperparameters"]["optimal_learning_rate"] = optimal_learning_rate

Computing optimal learning rate, iteration 1/100


/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:120: UserWarning: TypedStorage is deprecated. It will be removed in the future and

Computing optimal learning rate, iteration 2/100
Computing optimal learning rate, iteration 3/100
Computing optimal learning rate, iteration 4/100
Computing optimal learning rate, iteration 5/100
Computing optimal learning rate, iteration 6/100
Computing optimal learning rate, iteration 7/100
Computing optimal learning rate, iteration 8/100
Computing optimal learning rate, iteration 9/100
Computing optimal learning rate, iteration 10/100
Computing optimal learning rate, iteration 11/100
Computing optimal learning rate, iteration 12/100
Computing optimal learning rate, iteration 13/100
Computing optimal learning rate, iteration 14/100
Computing optimal learning rate, iteration 15/100
Computing optimal learning rate, iteration 16/100
Computing optimal learning rate, iteration 17/100
Computing optimal learning rate, iteration 18/100
Computing optimal learning rate, iteration 19/100
Computing optimal learning rate, iteration 20/100
Computing optimal learning rate, iteration 21/100
Computin

In [6]:
def get_train_dataloaders(dataset: torch.utils.data.Dataset):
    total_training_number = len(dataset)
    train_size = int(validation_split * total_training_number)
    test_size = total_training_number - train_size

    train_ds, val_ds = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True, num_workers=1
    )
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_loader, val_loader

In [ ]:
val_interval = 5

for percentage_data in [0.2, 0.4, 0.6, 0.8, 1.0]:
    config["hyperparameters"]["percentage_data"] = percentage_data

    train_data = ACDCDataset(
        data_dir=data_dir,
        train=True,
        transform=train_transforms,
        percentage_data=percentage_data,
    )
    train_loader, val_loader = get_train_dataloaders(train_data)

    wandb.init(
        project="acdc-3D-UNet-baseline", config=config["hyperparameters"], tags=["limited_data"], dir=log_dir,
        reinit=True
    )
    wandb.config.dataset = "ACDC"
    wandb.config.architecture = "UNet"

    out_dir = root_out_dir / f"percentage_data_{percentage_data}"
    os.makedirs(out_dir, exist_ok=True)

    epoch_loss_values, metric_values = train(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        loss_function=loss_function,
        optimizer=optimizer,
        val_interval=val_interval,
        epochs=epochs,
        metrics=metrics,
        device=device,
        out_dir=out_dir,
    )

    wandb.finish()

    # visualize_loss_curves(epoch_loss_values, metric_values, val_interval, out_dir)

wandb: Currently logged in as: josh-stein. Use `wandb login --relogin` to force relogin


----------
epoch 1/100


/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:120: UserWarning: TypedStorage is deprecated. It will be removed in the future and

1/16, train_loss: 0.8747
2/16, train_loss: 0.8703
3/16, train_loss: 0.8761
4/16, train_loss: 0.8704
5/16, train_loss: 0.8575
6/16, train_loss: 0.8600
7/16, train_loss: 0.8628
8/16, train_loss: 0.8472
9/16, train_loss: 0.8401
10/16, train_loss: 0.8435
11/16, train_loss: 0.8590
12/16, train_loss: 0.8322
13/16, train_loss: 0.8378
14/16, train_loss: 0.8359
15/16, train_loss: 0.8384
16/16, train_loss: 0.8256
epoch 1 average loss: 0.8520
----------
epoch 2/100


/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:


...

1/16, train_loss: 0.1193
2/16, train_loss: 0.0853
3/16, train_loss: 0.1190
4/16, train_loss: 0.1069
5/16, train_loss: 0.1723
6/16, train_loss: 0.1158
7/16, train_loss: 0.0947
8/16, train_loss: 0.1308
9/16, train_loss: 0.1124
10/16, train_loss: 0.0921
11/16, train_loss: 0.0954
12/16, train_loss: 0.1255
13/16, train_loss: 0.1132
14/16, train_loss: 0.1488
15/16, train_loss: 0.1154
16/16, train_loss: 0.1128
epoch 99 average loss: 0.1162
----------
epoch 100/100


/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:


1/16, train_loss: 0.1128
2/16, train_loss: 0.1295
3/16, train_loss: 0.1517
4/16, train_loss: 0.0853
5/16, train_loss: 0.0966
6/16, train_loss: 0.1432
7/16, train_loss: 0.1103
8/16, train_loss: 0.0970
9/16, train_loss: 0.1994
10/16, train_loss: 0.0859
11/16, train_loss: 0.1358
12/16, train_loss: 0.1000
13/16, train_loss: 0.1034
14/16, train_loss: 0.1786
15/16, train_loss: 0.1216
16/16, train_loss: 0.0976
epoch 100 average loss: 0.1218


/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/torch/_tensor.py:1295: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/Users/joshstein/Documents/thesis/code/.venv/lib/python3.11/site-packages/monai/data/__init__.py:127: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if storage.is_cuda:


current epoch: 100 current mean dice: 0.7249
best mean dice: 0.7312 at epoch: 65


In [ ]:
visualize_loss_curves(epoch_loss_values, metric_values, val_interval, out_dir)
for slice_no in [0, 2, 4]:
    visualize_predictions(
        model=model,
        model_file=os.path.join(out_dir, "best_metric_model.pth"),
        val_loader=val_loader,
        device=device,
        image_key="end_diastole",
        label_key="end_diastole_label",
        slice_no=slice_no,
    )